In [1]:
import os
import numpy as np
from PIL import Image
import face_recognition
import faiss

In [2]:
ROOTDATA = 'dataset'

def read_file(path):
    img_path = []
    for paths, subdirs, files in os.walk(path):
        for name in files:
            img_path.append(os.path.join(paths,name))
    return img_path


In [3]:
train_imgs = read_file(ROOTDATA)
train_imgs

['dataset\\Eimi_Fukada\\1.jpg',
 'dataset\\Eimi_Fukada\\2.jpg',
 'dataset\\Eimi_Fukada\\3.jpg',
 'dataset\\Eimi_Fukada\\4.jpg',
 'dataset\\Eimi_Fukada\\5.jpg',
 'dataset\\Eimi_Fukada\\6.jpg',
 'dataset\\Riho_Fujimori\\1.jpg',
 'dataset\\Riho_Fujimori\\2.jpg',
 'dataset\\Riho_Fujimori\\3.jpg',
 'dataset\\Riho_Fujimori\\4.jpg',
 'dataset\\Riho_Fujimori\\5.jpg',
 'dataset\\Riho_Fujimori\\6.jpg',
 'dataset\\Yua_Mikami\\1.jpg',
 'dataset\\Yua_Mikami\\2.jpg',
 'dataset\\Yua_Mikami\\3.jpg',
 'dataset\\Yua_Mikami\\4.jpg',
 'dataset\\Yua_Mikami\\5.jpg',
 'dataset\\Yua_Mikami\\6.jpg']

In [4]:
faceEncode = []
img_paths = []
for path in train_imgs:
    # read img
    img = face_recognition.load_image_file(path)
    # detect face
    img_location = face_recognition.face_locations(img)
    if len(img_location) > 0:
        #crop face
        for (top,right,bottom,left) in img_location:
            face_img = img[top:bottom,left:right]
            # save face img
            pil_img = Image.fromarray(face_img)
            pil_img.save(path)
            # encode
            face_encode = face_recognition.face_encodings(img)[0]
            faceEncode.append(face_encode)
            img_paths.append(path)
len(faceEncode)            

8

In [5]:
train_labels = np.array([img.split('\\')[-2] for img in img_paths])
len(train_labels)

8

In [6]:
len(img_paths)

8

In [7]:
faceEncode = np.array(faceEncode,dtype=np.float32)
faceEncode.shape

(8, 128)

In [8]:
# create index with faiss
face_index = faiss.IndexFlatL2(128)
# add vector
face_index.add(faceEncode)

In [15]:
#test
test_img = face_recognition.load_image_file('test/fukada.jpg')
img_location = face_recognition.face_locations(test_img)
if len(img_location) > 0:
    #crop face
    for (top,right,bottom,left) in img_location:
        face_img = img[top:bottom,left:right]
        qr = face_recognition.face_encodings(face_img)[0]
qr.shape        

(128,)

In [16]:
#convert to (n,128)
qr = np.array(qr,dtype=np.float32).reshape(-1,128)
qr.shape

(1, 128)

In [17]:
_,result = face_index.search(qr, k=5)
result

array([[15, 12, 14, 13, 10]], dtype=int64)

In [12]:
label = [train_labels[i] for i in result[0]]
label

array(['Eimi_Fukada', 'Eimi_Fukada', 'Eimi_Fukada', 'Riho_Fujimori',
       'Riho_Fujimori', 'Yua_Mikami', 'Yua_Mikami', 'Yua_Mikami'],
      dtype='<U13')

In [9]:
#save index
faiss.write_index(face_index,'face_index.bin')

In [13]:
np.save('labels.npy', train_labels) 